In [40]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
df=pd.read_csv('train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishekmahto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [9]:
df.shape

(20800, 5)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [13]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [15]:
df= df.fillna('')

In [17]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [19]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [21]:
df['content']=df['title']+ ' '+df['author']+ ' '+df['text']

In [23]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [25]:
x=df.drop(columns='label')
y=df['label']

In [27]:
print(x,y)

          id  ...                                            content
0          0  ...  House Dem Aide: We Didn’t Even See Comey’s Let...
1          1  ...  FLYNN: Hillary Clinton, Big Woman on Campus - ...
2          2  ...  Why the Truth Might Get You Fired Consortiumne...
3          3  ...  15 Civilians Killed In Single US Airstrike Hav...
4          4  ...  Iranian woman jailed for fictional unpublished...
...      ...  ...                                                ...
20795  20795  ...  Rapper T.I.: Trump a ’Poster Child For White S...
20796  20796  ...  N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797  20797  ...  Macy’s Is Said to Receive Takeover Approach by...
20798  20798  ...  NATO, Russia To Hold Parallel Exercises In Bal...
20799  20799  ...  What Keeps the F-35 Alive David Swanson   Davi...

[20800 rows x 5 columns] 0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

# Stemming:

# Stemming is the process of reducing a word to its Root word

# example: actor, actress, acting --> act



In [30]:
port_stem =PorterStemmer()

In [46]:
def stemming(content):
    stemming_content =re.sub('[^a-zA-Z)]',' ',content)
    stemming_content = stemming_content.lower()
    stemming_content = stemming_content.split()
    stemming_content = [port_stem.stem(word) for word in stemming_content if not word in stopwords.words('english')]
    stemming_content = ' '.join(stemming_content)
    return stemming_content

In [48]:
df['content']= df['content'].apply(stemming)

In [68]:
print(df['content'])

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2        truth might get fire consortiumnew com truth m...
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799    keep f aliv david swanson david swanson author...
Name: content, Length: 20800, dtype: object


In [70]:
x=df['content'].values
y=df['label'].values

In [72]:
print(x)

['hous dem aid even see comey letter jason chaffetz tweet darrel lucu hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common license) apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehouse) octob cours know case comey actual say review email light un

In [74]:
print(y)

[1 0 1 ... 0 1 1]


In [76]:
y.shape

(20800,)

In [80]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [82]:
print(x)

  (0, 330)	0.04347293629112574
  (0, 536)	0.022956480273882526
  (0, 656)	0.03868783674122817
  (0, 908)	0.015721198652448313
  (0, 964)	0.01637950467602218
  (0, 1355)	0.0210642891388746
  (0, 1678)	0.01798835878261526
  (0, 1959)	0.11535253281524613
  (0, 3126)	0.04658683372498751
  (0, 3147)	0.019003318691030772
  (0, 3489)	0.011787595105748004
  (0, 3865)	0.03084786370322493
  (0, 4288)	0.016666092941875404
  (0, 4397)	0.019407260950487108
  (0, 4448)	0.026397765933398185
  (0, 4737)	0.017684123455632313
  (0, 4926)	0.025930017667569485
  (0, 4977)	0.0416629775801125
  (0, 4995)	0.014388094555015
  (0, 6652)	0.018206494969359478
  (0, 7216)	0.020632042406134542
  (0, 8842)	0.021633914075679626
  (0, 9372)	0.014222696194301485
  (0, 11192)	0.047535980384920136
  (0, 12624)	0.027750421593183014
  :	:
  (20799, 110198)	0.0365795773597969
  (20799, 110337)	0.049199714127607735
  (20799, 110664)	0.0186552110520713
  (20799, 110700)	0.04313162229671856
  (20799, 111037)	0.017401704598728

In [84]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=2)

In [86]:
model = LogisticRegression()

In [88]:
model.fit(xtrain,ytrain)

LogisticRegression()

In [90]:
# accuracy score on the training data
X_train_prediction = model.predict(xtrain)
training_data_accuracy = accuracy_score(X_train_prediction, ytrain)

In [92]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9786057692307693


In [94]:
# accuracy score on the test data
X_test_prediction = model.predict(xtest)
test_data_accuracy = accuracy_score(X_test_prediction, ytest)

In [96]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9591346153846154


In [98]:
X_new = xtest[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [100]:
print(ytest[3])

1
